In [1]:
import numpy as np
import matplotlib.pyplot as plt
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf
from astropy.modeling import models, fitting
from scipy.ndimage import filters

In [2]:
infile1 = '../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0030-J0040+2555_LRISr_2020Nov21T045325.037.fits'
infile2 = '../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0031-J0040+2555_LRISr_2020Nov21T051705.971.fits'

In [3]:
spec1 = spec2d.Spec2d(infile1, hext=12, xtrim=[40, 170])
spec2 = spec2d.Spec2d(infile2, hext=12, xtrim=[40, 170])


Loading file ../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0030-J0040+2555_LRISr_2020Nov21T045325.037.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0030-J0040+2555_LRISr_2020Nov21T045325.037.fits (HDU=12)
The input dataset was trimmed
 xrange: 40:171.  yrange: 0:4096
Final data dimensions (x y): 131 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 131


Loading file ../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0031-J0040+2555_LRISr_2020Nov21T051705.971.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/lris-longslit-red-1120/spec2d_r201121_0031-J0040+2555_LRISr_2020Nov21T051705.971.fits (HDU=12)
The input dataset was trimmed
 xrange: 40:171.  yrange: 0:4096
Final data dimensions (x y): 131 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 131



In [4]:
spec1.set_dispaxis('y')
spec2.set_dispaxis('y')


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096




In [5]:
spec1.subtract_sky_2d(use_skymod=True)
spec2.subtract_sky_2d(use_skymod=True)


pypeit generated sky model will be used for sky subtraction

 Wrote sky subtracted data to None

From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


pypeit generated sky model will be used for sky subtraction

 Wrote sky subtracted data to None

From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00



In [7]:
sp_1 = spec1.data   #[:, 55:105]
sp_2 = spec2.data   #[:, 55:105]
sp_1.flatten().shape

(536576,)

In [ ]:
pf.PrimaryHDU(sp_1.T).writeto('sp1.fits')

In [ ]:
s1 =  spec1.data[:, 70:80]
s2 =  spec2.data[:, 70:80]

In [ ]:
%matplotlib notebook
count, bins, patch= plt.hist(s1.flatten(), bins=500)
counts, bins1, patch = plt.hist(s2.flatten(), bins=500)
#bins2= plt.hist(sp2.flatten(), bins=250)
#bins = plt.hist((sp2-sp1).flatten(), bins=200)
plt.yscale('log')
#print(count)
#print(counts)

In [8]:
## this is the code we are using if we have only two frames

mask = np.zeros(sp_1.shape, dtype=bool)

for k in range(0, sp_1.shape[0], 400):
    for i in range(0, sp_1.shape[1], 5):
        sp1 = sp_1[k:k+400, i:i+5]
        sp2 = sp_2[k:k+400, i:i+5]
        count1, bins1 = np.histogram(sp1.flatten(), bins=500)
        count2, bins2 = np.histogram(sp2.flatten(), bins=500)
        sm1 = 0
        sm2 = 0
        cnt1 = 0
        cnt2 = 0
        tot_pix =sp1.flatten().shape[0]

        for j,p in enumerate(count1):
            if p > cnt1:
                cnt1 = p
                max_bin1 = bins1[j+1]
            sm1 += p
            if sm1/tot_pix > 0.95 :
                border1 = 0.5*(bins1[j+1] + bins1[j])
                break
        #print(border)
        for j,p in enumerate(count2):
            if p > cnt2:
                cnt2 = p
                max_bin2 = 0.5*(bins2[j+1] + bins2[j])
            sm2 += p
            if sm2/tot_pix > 0.95 :
                border2 = bins2[j+1]
                break

        a = np.where(sp1>border1)
        a =np.transpose(a)
        
        if abs(border1 - max_bin1) < 200 :
            mp = 200    #/ abs(border1 - max_bin1) 
        elif abs(border1 - max_bin1) < 500:
            mp = 300
        else:
            mp = 400

        for j, p in enumerate(a):

            #print(sp2[p[0]][p[1]])
            if sp2[p[0]][p[1]] >0 :
                #pass 
                if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]] + mp) :
                    mask[k+p[0]][i+p[1]] =1 
            else:
                if sp1[p[0]][p[1]] > 3*border1 :
                    mask[k+p[0]][i+p[1]] =1

In [ ]:
m = sp1 < 856
np.sum(m)/tot_pix

In [9]:
 sp_1[mask] = sp_2[mask]

In [ ]:
sp_1[mask] = -50000

In [10]:
pf.PrimaryHDU(sp_1.T).writeto('sp1_fill_new1.fits')

In [57]:
m =np.where(mask==1)
m = np.transpose(m)
print(m)
for i,p in enumerate(m[0:1]):
    a = sp_1[p[0]-20:p[0]+20, p[1]:p[1]+1]
    ind = [[i, p[1]] for i in range(p[0]-20, p[0]+20, 1)]
    #print(a)
    print(ind)
    for j, r in enumerate(ind):
        #print(r)
        if r in m.tolist():
            print(r)

[[  59   60]
 [  59   61]
 [  60   60]
 ...
 [4095   75]
 [4095   76]
 [4095   77]]
[[39, 60], [40, 60], [41, 60], [42, 60], [43, 60], [44, 60], [45, 60], [46, 60], [47, 60], [48, 60], [49, 60], [50, 60], [51, 60], [52, 60], [53, 60], [54, 60], [55, 60], [56, 60], [57, 60], [58, 60], [59, 60], [60, 60], [61, 60], [62, 60], [63, 60], [64, 60], [65, 60], [66, 60], [67, 60], [68, 60], [69, 60], [70, 60], [71, 60], [72, 60], [73, 60], [74, 60], [75, 60], [76, 60], [77, 60], [78, 60]]
[59, 60]
[60, 60]


In [45]:
a =[[5, 6],
    [7,8]]

if [6,5] in a:
    print('yes')
m    

array([[  59,   60],
       [  59,   61],
       [  60,   60],
       ...,
       [4095,   75],
       [4095,   76],
       [4095,   77]])

In [ ]:
a = np.where(sp1>border)
mask = np.zeros(sp1.shape, dtype=bool)
a =np.transpose(a)
for i, p in enumerate(a):
    
    #print(sp2[p[0]][p[1]])
    if sp2[p[0]][p[1]] >0 :
        #pass 
        if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]] + 250) :
            mask[p[0]][p[1]] =1 
    else:
        if sp1[p[0]][p[1]] > border + 300 :
            mask[p[0]][p[1]] =1 
    
    #print(p)
    #pass
#a[0]
#mask[0][0] = 1
np.where(mask)

In [ ]:
m = sp_1 == -50000
sp_1[m] = sp_2[m]

In [ ]:
pf.PrimaryHDU(sp_1.T).writeto('sp1_fill.fits')

In [ ]:
diff = spec1.data - spec2.data
#pf.PrimaryHDU(diff.T).writeto('diff1.fits')

In [ ]:
mask1 = diff_md < -30 
mask2 = diff_md > 30

mask = mask1 | mask2

In [ ]:
np.sum(1*mask)